In [1]:
# Import common libraries
import numpy as np
import pandas as pd
import os

# Import MNE processing
import mne
from mne.preprocessing.nirs import optical_density, beer_lambert_law

# Import MNE-NIRS processing
from mne_nirs.statistics import run_glm
from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import statsmodels_to_results
from mne_nirs.channels import get_short_channels, get_long_channels
from mne_nirs.channels import picks_pair_to_idx
from mne_nirs.visualisation import plot_glm_group_topo
from mne_nirs.datasets import fnirs_motor_group
from mne_nirs.visualisation import plot_glm_surface_projection
from mne_nirs.io.fold import fold_channel_specificity

# Import MNE-BIDS processing
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals

# Import StatsModels
import statsmodels.formula.api as smf

# Import Plotting Library
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [2]:
# import raw nirx data
root = "../BIDS_Anon/"
# raw_intensity = mne.io.read_raw_snirf(fname, verbose=True, preload=True)
# raw_intensity.plot_sensors()

dataset = BIDSPath(root=root, task="AnonCom", datatype="nirs", suffix="nirs", extension=".snirf")

print(dataset.directory)

../BIDS_Anon/nirs


In [3]:
def individual_analysis(bids_path, ID):
    # print(bids_path, ID)
    raw_intensity = mne.io.read_raw_snirf(bids_path, verbose=True, preload=True)
    
    raw_intensity.annotations.rename({'4': 'Control', '2': 'Neutral', '3': 'Inflammatory', '1':'Practice'})

    # Get Events
    events, event_dict = mne.events_from_annotations(raw_intensity, verbose=False)
    
    # Rename Events
    # event_dict = {'Control': 4, 'Neutral': 2, 'Inflammatory': 3, 'Practice': 1}
    
    # Signal Processing
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
    # print("Raw OD", raw_od)
    
    # Convert signal to haemoglobin and resample
    raw_haemo = beer_lambert_law(raw_od, ppf=0.1)
    raw_haemo.resample(0.3)
    # print("Raw Haemo", raw_haemo)
    
    # Make First Level Design Matrix
    design_matrix = make_first_level_design_matrix(raw_haemo, drift_model='cosine', high_pass=0.005, hrf_model='spm', stim_dur=30.0)
    

    # Run GLM
    glm_est = run_glm(raw_haemo, design_matrix)
    
    # Calculate Values of interest
    cha = glm_est.to_dataframe()
    
    # Compute defined contrast
    # print(design_matrix.columns)

    contrast_matrix = np.eye(design_matrix.shape[1])
    basic_conts = dict([(column, contrast_matrix[i])
                        for i, column in enumerate(design_matrix.columns)])
    neutral = basic_conts['Control'] - basic_conts['Neutral']
    inflam = basic_conts['Control'] - basic_conts['Inflammatory']
    
    con_inflam = glm_est.compute_contrast(inflam)
    con_neutral = glm_est.compute_contrast(neutral)
    
    neutral_contrast = con_neutral.to_dataframe()
    inflam_contrast = con_inflam.to_dataframe()


    return raw_haemo, cha, neutral_contrast, inflam_contrast

#  cha = glm_est.to_dataframe()
#  roi = glm_est.to_dataframe_region_of_interest(groups, design_matrix.columns, demographic_info=True)
# contrast = glm_est.compute_contrast(contrast_LvR)
#     con = contrast.to_dataframe()
# raw_haemo

In [10]:
df_roi = pd.DataFrame()  # To store region of interest results
df_cha = pd.DataFrame()  # To store channel level results
df_con_neutral = pd.DataFrame()  # To store channel level neutral contrast results
df_con_inflam = pd.DataFrame()  # To store channel level inflam contrast results

subjects = os.listdir("../BIDS_Anon/")
for sub in subjects:
    if sub != ".DS_Store":
        # Create path to file based on experiment info
        f_path = f'../BIDS_Anon/{sub}/nirs/{sub}_task-AnonCom_nirs.snirf'
        
        # Analyse data and return both ROI and channel results
        raw_haemo, cha, neutral_contrast, inflam_contrast = individual_analysis(f_path, sub)
        
        # Append individual results to all participants
        # df_roi = pd.concat([df_roi, roi], ignore_index=True)
        df_cha = pd.concat([df_cha, cha], ignore_index=True)
        df_con_neutral = pd.concat([df_con_neutral, neutral_contrast], ignore_index=True)
        df_con_inflam = pd.concat([df_con_inflam, inflam_contrast], ignore_index=True)
        
df_con_neutral

Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Anon/sub-05/nirs/sub-05_task-AnonCom_nirs.snirf
Reading 0 ... 17061  =      0.000 ...  1677.096 secs...
Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Anon/sub-03/nirs/sub-03_task-AnonCom_nirs.snirf
Reading 0 ... 22349  =      0.000 ...  2196.907 secs...


variable,ContrastType,effect,p_value,stat,z_score,Source,Detector,Chroma,Significant,ch_name
0,t,-1.350161e-05,0.526075,-0.065441,-0.065407,1,1,hbo,False,S1_D1 hbo
1,t,1.590763e-05,0.463165,0.092513,0.092464,1,2,hbo,False,S1_D2 hbo
2,t,3.150076e-06,0.493957,0.015157,0.015149,2,1,hbo,False,S2_D1 hbo
3,t,-2.764459e-05,0.548060,-0.120824,-0.120760,2,3,hbo,False,S2_D3 hbo
4,t,4.104869e-05,0.213627,0.794571,0.793899,3,2,hbo,False,S3_D2 hbo
...,...,...,...,...,...,...,...,...,...,...
75,t,-1.295326e-06,0.610541,-0.280850,-0.280730,6,6,hbr,False,S6_D6 hbr
76,t,-3.920394e-06,0.836204,-0.979734,-0.978976,7,5,hbr,False,S7_D5 hbr
77,t,2.731955e-06,0.274137,0.600670,0.600348,7,7,hbr,False,S7_D7 hbr
78,t,-6.708587e-07,0.560465,-0.152211,-0.152149,8,6,hbr,False,S8_D6 hbr


In [5]:
# Compute Group Level Results

